In [ ]:
                                              #in the name of creator Ali



import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# adress
base_url = "https://www.dadrah.ir/"
list_url = "https://www.dadrah.ir/consulting-catalog.php?page="

# Header settings for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# lists for save data
data = []

# max of lops
max_retries = 5


#Number of pages to process between each save

save_interval = 100  

# Circle to navigate different pages of the site

for page in range(1,2):                            # Change to the number of  pages you want
    for attempt in range(max_retries):
        try:
           #request to website
            response = requests.get(list_url + str(page), headers=headers, timeout=10)
            time.sleep(2) 
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                #  export date
                calendar_icons = soup.find_all('i', class_='fa fa-calendar')
                for calendar_icon in calendar_icons:
                    date_span = calendar_icon.find_parent('span', class_='pull-right')
                    if date_span:
                        persian_date_text = date_span.text.strip()
                    else:
                        persian_date_text = "Date not found"

                # link and element
                question_links = soup.find_all('a', href=True, string="ادامه ی مطلب و پاسخ ها ...")
                
                for link in question_links:
                    question_url = base_url + link['href']  # ساخت URL کامل برای دسترسی به صفحه سوال
                    question_response = requests.get(question_url, headers=headers, timeout=10)
                    
                    if question_response.status_code == 200:
                        question_soup = BeautifulSoup(question_response.text, 'html.parser')
                        
                        # subject
                        question_subject = question_soup.find('a', class_='btn btn-info tags')
                        subject_text = question_subject.text.strip() if question_subject else "Not found"
                        
                        # question
                        full_question = question_soup.find('p', attrs={'style': 'text-align:justify;'})
                        question_text = full_question.text.strip() if full_question else "Not found"
                        
                        # answers
                        answers = question_soup.find_all('p', class_='text-justify text-primary')
                        for answer in answers:
                            answer_text = answer.text.strip()
                            
                            # dictionary
                            data.append({
                                "Subject": subject_text,
                                "Question": question_text,
                                "Question Date": persian_date_text,
                                "Answer": answer_text,
                            })
                    else:
                        print(f"Failed to retrieve the question page: {question_url}")
                break  
            else:
                print(f"Error: Could not retrieve page {page}")
        except requests.exceptions.RequestException as e:
            print(f"Error on page {page}, attempt {attempt+1}: {e}")
            time.sleep(5)  
    else:
        print(f"Failed to retrieve page {page} after {max_retries} attempts")

    # Save data at specified intervals
    if page % save_interval == 0:
        df = pd.DataFrame(data)
        df.to_excel(f"extracted_data_temp_page_{page}.xlsx", index=False, engine='openpyxl')
        print(f"Data saved after processing {page} pages")

# final save
df = pd.DataFrame(data)
df.to_excel("extracted_data_with_subject.akhar123.xlsx", index=False, engine='openpyxl')

print("Final data has been successfully saved to extracted_data_with_subject.akhar.xlsx")


#                                             رندان تشنه لب را آبی نمیدهد کس  
#                                            گویی ولی شناسان رفتند از این ولایت
